# Daily availability data analysis

In [1]:
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

In [2]:
DATA_DIR = Path("../daily/")

In [3]:
def concat_all_dailies():
    dfs = []
    
    for loc in DATA_DIR.glob("*.csv"):
        df = pd.read_csv(loc)
        df = df.rename(columns={df.columns[0]: "name"})

        date = pd.to_datetime(loc.stem)
        df.insert(1, "date", date)  # Insert date as the second column
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

In [4]:
df = concat_all_dailies()
df.head()

,name,date,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,...,2024-09-19,2024-09-20,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28
0,Chalet Les Méandres (ex Tupilak),2024-06-26,12,23,22,21,22,17,11,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gîte Michel Fagot,2024-06-26,0,0,0,0,0,0,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Refuge du Fioux,2024-06-26,3,16,18,18,10,10,17,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Auberge du Truc,2024-06-26,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,La Ferme à Piron,2024-06-26,0,0,0,5,0,0,8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## How many bookable beds have there been?
TMB huts are notoriously hard to book. 
There are several stages of the TMB where there are not enough huts for the number of people that traverse the tour.
Two examples are Les Chapieux (Auberge de la Nova, Les Chambres du Soleil, Refuge des Mottets) and Trient (Hôtel du Col de la Forclaz, Hôtel La Grande Ourse, Refuge Le Peuty, Auberge Mont-Blanc).
Can we use our daily availability data to show how many hard it really is? Specifically, how many beds could be booked in a given date range for a given hut and booking date?

Let's define this data problem.
- $H$: Set of huts.
- $B$: Set of booking dates (i.e., the hiking season).
- $D$: Set of availability fetching dates.
- $n^{hb}_d$: Number of available beds for hut $h$ for booking date $b$ accessed on date $d$.

Consider a hut $h$ and a booking date $b$.
Let $s$ denote the start date of interest and let $e$ denote the end date of interest with $s \le e \le b$.
Define $\Delta_{d}^{hb} = n_{d}^{hb} - n_{d-1}^{hb}$ as the change in the number of beds between two access dates; a positive value means that beds have become available.
Then $$\sum_{d=s}^{e} \max \{\Delta_d^{hb}, 0\}$$ calculates the number of bookable beds for the problem.


## Data preparation
We have to prepare the data first. 
Here we order the availablility data on name and date.

In [5]:
on_name_date = df.sort_values(['name', 'date'])
on_name_date.head()

,name,date,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,...,2024-09-19,2024-09-20,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28
8121,Auberge Gîte Bon Abri,2023-12-11,7,7,7,7,7,7,15,32,...,57.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8681,Auberge Gîte Bon Abri,2023-12-12,7,7,7,7,7,7,15,32,...,57.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8401,Auberge Gîte Bon Abri,2023-12-13,7,7,7,7,7,7,15,32,...,57.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7321,Auberge Gîte Bon Abri,2023-12-14,7,7,7,7,7,7,15,32,...,57.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6881,Auberge Gîte Bon Abri,2023-12-15,7,7,7,7,7,7,15,32,...,57.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


It's much easier to work with rows, so we transform the booking date columns into rows.

In [6]:
booking_dates = [c for c in on_name_date.columns if c not in ["name", "date"]]
melted = pd.melt(on_name_date, id_vars=["name", "date"], value_vars=booking_dates, var_name="booking_date")
melted.head()

,name,date,booking_date,value
0,Auberge Gîte Bon Abri,2023-12-11,2024-06-01,7.0
1,Auberge Gîte Bon Abri,2023-12-12,2024-06-01,7.0
2,Auberge Gîte Bon Abri,2023-12-13,2024-06-01,7.0
3,Auberge Gîte Bon Abri,2023-12-14,2024-06-01,7.0
4,Auberge Gîte Bon Abri,2023-12-15,2024-06-01,7.0


Grouping by name and booking date gives us a convenient overview of all fetched availability sorted on date.

In [7]:
by_name_booking = melted.groupby(["name", "booking_date"])
by_name_booking.get_group(('Auberge Gîte Bon Abri', '2024-06-05')).head()

,name,date,booking_date,value
35072,Auberge Gîte Bon Abri,2023-12-11,2024-06-05,7.0
35073,Auberge Gîte Bon Abri,2023-12-12,2024-06-05,7.0
35074,Auberge Gîte Bon Abri,2023-12-13,2024-06-05,7.0
35075,Auberge Gîte Bon Abri,2023-12-14,2024-06-05,7.0
35076,Auberge Gîte Bon Abri,2023-12-15,2024-06-05,7.0


Now we perform the following calculations on each group:
- Compute the differences between each two rows. For the first row, we assume that the previous row was zero.
- Sum all positive differences up to the booking date of the group.

In [8]:
def compute_bookable_beds(group):
    name, booking_date = group.name
    booking_date = datetime.strptime(booking_date, "%Y-%m-%d")

    sub = group[group['date'].dt.date <= booking_date.date()]
    changes = np.diff(sub['value'], prepend=0)
    return sum(changes[changes > 0])

In [9]:
bookable = by_name_booking.apply(compute_bookable_beds).reset_index().rename(columns={0: "value"})
bookable.head()

,name,booking_date,value
0,Auberge Gîte Bon Abri,2024-06-01,31.0
1,Auberge Gîte Bon Abri,2024-06-02,31.0
2,Auberge Gîte Bon Abri,2024-06-03,31.0
3,Auberge Gîte Bon Abri,2024-06-04,31.0
4,Auberge Gîte Bon Abri,2024-06-05,31.0


In [10]:
df_wide = bookable.pivot(index='name', columns='booking_date', values='value')
df_wide.head(10)

booking_date,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,2024-06-06,2024-06-07,2024-06-08,2024-06-09,2024-06-10,...,2024-09-19,2024-09-20,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28
name,,,,,,,,,,,,,,,,,,,,,
Auberge Gîte Bon Abri,31.0,31.0,31.0,31.0,31.0,31.0,34.0,51.0,39.0,42.0,...,57.0,41.0,57.0,57.0,57.0,57.0,57.0,57.0,57.0,57.0
Auberge Mont-Blanc,28.0,24.0,32.0,32.0,32.0,27.0,32.0,32.0,16.0,17.0,...,11.0,10.0,26.0,14.0,32.0,28.0,28.0,28.0,0.0,0.0
Auberge des Glaciers,0.0,0.0,52.0,52.0,52.0,52.0,52.0,50.0,52.0,52.0,...,52.0,52.0,58.0,52.0,52.0,52.0,52.0,52.0,52.0,52.0
Auberge du Truc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Auberge la Boërne,0.0,1.0,29.0,31.0,27.0,33.0,33.0,33.0,33.0,29.0,...,38.0,12.0,28.0,32.0,30.0,19.0,31.0,31.0,31.0,31.0
Auberge-Refuge de la Nova,0.0,0.0,0.0,0.0,0.0,0.0,52.0,36.0,59.0,45.0,...,25.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabane du Combal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chalet 'Le Dolent',0.0,0.0,30.0,30.0,30.0,30.0,28.0,0.0,0.0,30.0,...,30.0,29.0,0.0,0.0,30.0,30.0,30.0,30.0,30.0,0.0
Chalet La Grange,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_wide.to_csv("../tmp/bookable.csv", encoding='utf-8')